In [2]:
import os
import cv2
import numpy as np
import random
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

In [ ]:
class newModel:

    def __init__(self, target_x, target_y, folder, model, resize_function, chunk_size=100):
        self.target_x = target_x
        self.target_y = target_y
        self.folder = folder
        self.model = model
        self.resize_function = resize_function
        self.chunk_size = chunk_size
        self.labels = []

    def apply_features(self, img):
        '''Apply different features such as: Edge Filters, garbor filters, etc...'''
        return img

    def resize(self, img):
        ''' Output will change based on resize function (Bilinear, Randomized Padding, etc.)'''
        return img
    
    def augment(self, img):
        return img
    
    def gen_labels(self, files):
        for filename in files:
            label = filename.split('.')[0]
            self.labels.append(label)
            
        self.labels = to_categorical(self.labels)
        
    
    def train(self, images, labels):
        images = images.astype('float32') / 255.0
        self.model.fit(images, labels)

    def load_images_and_train(self):
        '''Function to load in images from a folder.'''
        images = []
        labels = []
        files = os.listdir(self.folder)

        files = random.shuffle(files)

        self.gen_labels(files)

        for i, filename in enumerate(files):
            if filename.endswith('.jpg'):
                img_path = os.path.join(self.folder, filename)
                img = load_img(img_path, target_size=(self.target_x, self.target_y))  # Resize images if necessary

                img = self.resize(img) # use the current resize function for this iteration.

                img = self.augment(img) # Create different augmentations of the image.

                img = self.apply_features(img) # TODO: Make this a list of images or image arrays

                img_array = img_to_array(img) # TODO: Convert list of images to multi-dimension array of images.
                images.append(img_array) # Append Image with its filters to images 

                labels.append(self.labels[i])
            if i//self.chunk_size == 0:
                # Train current array and labels
                self.train(np.array(images), np.array(labels))
                
                # Empty out variables
                images, labels = [], []
        
        if True: # Check if labels list is empty.
            # Train current array and labels
            self.train(np.array(images), np.array(labels))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

target_x

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(target_x, target_y, features)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Replace `num_classes` with the actual number of classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])